<a href="https://colab.research.google.com/github/thanecutler/pokemon-set-data-price-scraper/blob/main/pokemon_set_data_scrape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [174]:
# imports

import requests
import re
import json
import time
from datetime import date
from bs4 import BeautifulSoup
from IPython.display import clear_output

In [175]:
# sets and url stem

# sets = soup.findAll('select', 'priceGuideDropDown')[1]
# options = sets.findAll('option')

# set_name_list = []

# for option in options:
#   set_name_list.append(option['value'])

# print(set_name_list)

pokemon_url = 'https://shop.tcgplayer.com/price-guide/pokemon/'
pokemon_sets = ['swsh09-brilliant-stars', 'swsh09-brilliant-stars-trainer-gallery', 'swsh08-fusion-strike', 'celebrations', 'celebrations-classic-collection', 'swsh07-evolving-skies', 'swsh06-chilling-reign', 'swsh05-battle-styles', 'first-partner-pack', 'shining-fates', 'shining-fates-shiny-vault', 'mcdonalds-25th-anniversary-promos', 'swsh04-vivid-voltage', 'champions-path', 'swsh03-darkness-ablaze', 'battle-academy', 'swsh02-rebel-clash', 'swsh01-sword-and-shield-base-set', 'swsh-sword-and-shield-promo-cards', 'sm-cosmic-eclipse', 'mcdonalds-promos-2019', 'hidden-fates', 'hidden-fates-shiny-vault', 'sm-unified-minds', 'sm-unbroken-bonds', 'detective-pikachu', 'sm-team-up', 'mcdonalds-promos-2018', 'sm-lost-thunder', 'miscellaneous-cards-and-products', 'dragon-majesty', 'sm-celestial-storm', 'world-championship-decks', 'sm-forbidden-light', 'sm-trainer-kit-alolan-sandslash-and-alolan-ninetales', 'sm-ultra-prism', 'mcdonalds-promos-2017', 'sm-crimson-invasion', 'shining-legends', 'sm-burning-shadows', 'alternate-art-promos', 'sm-guardians-rising', 'sm-trainer-kit-lycanroc-and-alolan-raichu', 'sm-base-set', 'sm-promos', 'xy-evolutions', 'deck-exclusives', 'xy-steam-siege', 'league-and-championship-cards', 'xy-fates-collide', 'xy-trainer-kit-pikachu-libre-and-suicune', 'generations', 'generations-radiant-collection', 'xy-breakpoint', 'mcdonalds-promos-2015', 'xy-breakthrough', 'xy-ancient-origins', 'xy-roaring-skies', 'xy-trainer-kit-latias-and-latios', 'jumbo-cards', 'double-crisis', 'xy-primal-clash', 'xy-trainer-kit-bisharp-and-wigglytuff', 'xy-phantom-forces', 'xy-furious-fists', 'mcdonalds-promos-2014', 'xy-flashfire', 'xy-trainer-kit-sylveon-and-noivern', 'xy-base-set', 'xy-promos', 'legendary-treasures', 'legendary-treasures-radiant-collection', 'plasma-blast', 'plasma-freeze', 'plasma-storm', 'boundaries-crossed', 'dragons-exalted', 'dark-explorers', 'noble-victories', 'bw-trainer-kit-excadrill-and-zoroark', 'emerging-powers', 'black-and-white', 'call-of-legends', 'professor-program-promos', 'triumphant', 'undaunted', 'pikachu-world-collection-promos', 'unleashed', 'hgss-promos', 'arceus', 'supreme-victors', 'burger-king-promos', 'countdown-calendar-promos', 'legends-awakened', 'great-encounters', 'dp-trainer-kit-manaphy-and-lucario', 'diamond-and-pearl', 'diamond-and-pearl-promos', 'dragon-frontiers', 'crystal-guardians', 'delta-species', 'unseen-forces', 'emerald', 'deoxys', 'ex-battle-stadium', 'kids-wb-promos', 'team-magma-vs-team-aqua', 'dragon', 'aquapolis', 'best-of-promos', 'team-rocket', 'fossil', 'wotc-promo', 'blister-exclusives', 'base-set', 'base-set-shadowless', 'base-set-2', 'black-and-white-promos', 'dp-training-kit-1-blue', 'dp-training-kit-1-gold', 'dragon-vault', 'ex-trainer-kit-1-latias-and-latios', 'ex-trainer-kit-2-plusle-and-minun', 'expedition', 'firered-and-leafgreen', 'gym-challenge', 'gym-heroes', 'heartgold-soulsilver', 'hgss-trainer-kit-gyarados-and-raichu', 'hidden-legends', 'holon-phantoms', 'jungle', 'kalos-starter-set', 'legend-maker', 'legendary-collection', 'majestic-dawn', 'mcdonalds-promos-2011', 'mcdonalds-promos-2012', 'mysterious-treasures', 'neo-destiny', 'neo-discovery', 'neo-genesis', 'neo-revelation', 'next-destinies', 'nintendo-promos', 'platinum', 'pop-series-1', 'pop-series-2', 'pop-series-3', 'pop-series-4', 'pop-series-5', 'pop-series-6', 'pop-series-7', 'pop-series-8', 'pop-series-9', 'power-keepers', 'rising-rivals', 'ruby-and-sapphire', 'rumble', 'sandstorm', 'secret-wonders', 'skyridge', 'southern-islands', 'stormfront', 'team-rocket-returns']

In [181]:
# get prices function

def scrape_data(url, set_list):

  print('Beginning collection...')

  set_price_list = {'data': []}
  skipped_sets = []

  for ind, set_name in enumerate(set_list):

    clear_output(wait=True)

    try: 
      page_raw = requests.get(url + set_name)
      soup = BeautifulSoup(page_raw.text, 'html.parser')
    except:
      'Error occured retrieving data'
      skipped_sets.append(set_name)

    # create list of cells with price data in them
    price_td = soup.find_all('td', class_='marketPrice')

    # create new list to aggregate prices
    price_array = []

    # loop through prices, push to array
    # format: 0000.00
    for price in price_td:
      price = re.findall('([0-9,]*\.[0-9]*)', price.text)
      if len(price):
        price_array.append(float(price[0].replace(',', '')))

    # calculate total of price_array, round to two decimals
    total = round(sum(map(float, price_array)), 2)
    
    # create new dict to push to set_price_list
    set_obj = {'set_name': set_name.replace('-', ' ').title(), 'value': total, 'url': url + set_name, 'card_count': len(price_td)}
    set_price_list['data'].append(set_obj)

    print('Completed {} of {} sets'.format(ind + 1, len(set_list)))

    # wait ten seconds for crawl delay
    if (ind + 1 != len(set_list)):
      print('Waiting 10 seconds for next set...')
      time.sleep(10)

  today = date.today().strftime('%Y-%m-%d')
  set_price_list['date'] = today
  
  file_title = 'pokemon-set-price-data-{}.json'.format(today)

  # output to results.json file
  with open(file_title, 'w') as file:
    json.dump(set_price_list, file)

  # write to log file
  with open('logs.txt', 'a') as file:
    file.write('Finished gathering data. Skipped {} sets. Results saved in {}\n'.format(len(skipped_sets),file_title))
    if len(skipped_sets):
      file.write('\nSkipped sets:')
      for set_name in skipped_sets:
        file.write('\n\t' + set_name)
      file.write('\n')

  print('Finished gathering data. Skipped {} sets. Results saved in {}'.format(len(skipped_sets),file_title))

In [180]:
scrape_data(pokemon_url, pokemon_sets[slice(0,2)])

Completed 2 of 2 sets
Finished gathering data. Skipped 0 sets. Results saved in pokemon-set-price-data-2022-03-05.json
